In [ ]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

from IPython.display import display

%matplotlib inline

# Problem Description
The NCAA Basketball tournament is a time honored tradition that happens every March. Every year it earns it's nickname "March Madness" because of the upsets and the disappointments of people all around the world attempting to predict which team will win it all.

With this projects we're attempting to leverage the NCAA data and data science / machine learning to do just that, predict the tournament champion.

This Kaggle competition is scored based on the log-loss function.

## Log-Loss
Log-Loss is defined as $$-\frac{1}{n}\sum_{i=1}^n[y_{i}\log(y_{\hat{i}})+(1-y)\log(1-y_{\hat{i}})$$

* n is the number of games played
* $y_{\hat{i}}$ is the predicted probability of team1 beating team2
* $y_{i}$ is 1 if team1 wins, otherwise 0 if team2 wins
* $\log()$ is the natural (base e) logarithm

The use of the logarithm provides extreme punishments for being both confident and wrong. In the worst possible case, a prediction that something is true when it is actually false will add an infinite amount to your error score. In order to prevent this, predictions are bounded away from the extremes by a small value.

For reference a coin flip (50/50) would give us a log-loss ratio of just over 0.69. Our goal is to get under this number and get it as close to 0 as possible.

In [ ]:
N = 10000
r = [i/N for i in range(1,N)]
t = [1-ri for ri in r]
f = [ri for ri in r]

tll = [log_loss([0], list(zip([1-ri], [ri])), labels=[0, 1]) for ri in r]
fll = [log_loss([0], list(zip([ri], [1-ri])), labels=[0, 1]) for ri in r]

The following graph helps illustrate how log-loss works.

In [ ]:
fig, ax = plt.subplots()
ax.plot(tll, label='Actual Value: 0')
ax.plot(fll, label='Actual Value: 1')
ax.axvline(N/2, color='k', linestyle='--', label='Coin Flip')
plt.title('Log-Loss Values')
plt.xlabel('Predicted Probability')
plt.ylabel('Log Loss Score')
plt.legend()
xtl = [xt/N for xt in ax.get_xticks().tolist()]
ax.set_xticklabels(xtl);

# Limitations
- In the interest of time and simplicity we will ignore the players and events as part of this year's model. It could be expanded on at a later time to enchance the model with this additional data.
- Seeds will not be used as they are a biased ranking

# Import Data
Kaggle & Google kindly supply us with data surrounding the teams, their stats and coaches each year. We are leveraging that data below.

In [ ]:
data_dir = 'data_2019'

### Teams

In [ ]:
df_teams = pd.read_csv(os.path.join(data_dir, 'Teams.csv'))
df_teams.head()

We're going to engineer our first additional feature by calculating the number of seasons the school has been in the D1 conference.

In [ ]:
df_teams['D1Seasons'] = df_teams['LastD1Season'] - df_teams['FirstD1Season']
teams_dict = df_teams[['TeamID', 'TeamName']].to_dict()
del df_teams['TeamName']
df_teams.head()

### Detailed Game Statistics

In [ ]:
df_detailed = pd.read_csv(os.path.join(data_dir, 'Prelim2019_RegularSeasonDetailedResults.csv'))
df_detailed.head()

Here we calculate the difference in score which is a slightly better metric because if a team consistently scores 80+ points but allows their opponent to score 80+ points it's not as resounding a win or loss as if the opposing team scored only 40 points.

In [ ]:
df_detailed['ScoreDiff'] = df_detailed['WScore'] - df_detailed['LScore']

Here we pull only the winning team columns and adding 1 to the W(in) colums and 0 to the L(oss) column.

In [ ]:
ignore_cols = ['WLoc']
include_cols = ['DayNum', 'Season']
w_cols = [c for c in df_detailed 
          if (c.startswith('W') and c not in ignore_cols) or 
              c in include_cols]
df_stats_W = df_detailed.loc[:, w_cols]
c_names = {c:(c[1:] if c not in include_cols else c) for c in df_stats_W}
df_stats_W.rename(columns=c_names, inplace=True)
df_stats_W['W'] = 1
df_stats_W['L'] = 0
df_stats_W.head()

We do the same thing for all of the losses. This allows us to track a team's record throughout the season.

In [ ]:
ignore_cols = ['WLoc']
include_cols = ['DayNum', 'Season']
cols = [c for c in df_detailed if (c.startswith('L') and c not in ignore_cols) or c in include_cols]
df_stats_L = df_detailed.loc[:, cols]
c_names = {c:(c[1:] if c not in include_cols else c) for c in df_stats_L}
df_stats_L.rename(columns=c_names, inplace=True)
df_stats_L['W'] = 0
df_stats_L['L'] = 1
df_stats_L.head()

Now we combine all of the wins and losses together so we have one data source for a team's record.

In [ ]:
df_stats = df_stats_W.append(df_stats_L)
df_stats = df_stats.sort_values(by=['TeamID', 'Season', 'DayNum'])
df_stats.head()

We're engineering another feature here for the game number. This will allow us to track a teams game average stats as the season progresses. (Group by and then run the cumulative sum)

In [ ]:
df_c_stats = df_stats.groupby(['TeamID', 'Season', 'DayNum']).sum().groupby(level=[0, 1]).cumsum()
df_c_stats.reset_index(inplace=True)
df_c_stats['GameNum'] = df_c_stats.reset_index().groupby(['TeamID', 'Season']).cumcount() + 1
df_c_stats.set_index(['TeamID', 'Season'], inplace=True)
df_c_stats.head()

Calculate the percentages and per game stats

In [ ]:
df_c_stats['FG_PCT'] = df_c_stats['FGM'] / df_c_stats['FGA']
df_c_stats['FG3_PCT'] = df_c_stats['FGM3'] / df_c_stats['FGA3']
df_c_stats['FT_PCT'] = df_c_stats['FTM'] / df_c_stats['FTA']

PGL = ['Score', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
for pg in PGL:
    df_c_stats[f'{pg}_PG'] = df_c_stats[pg] / df_c_stats['GameNum']
df_c_stats.head()

Since Team1 v Team2 is the same as Team2 v Team1 the Kaggle competition requires the lower team ID be team1 and the higher ID team be team2 for predition purposes. The predition is that Team1 will beat Team2.

Here we're putting the prediction (outcome variable) in a format of 1 or 0 for the model to best utilize.

In [ ]:
df_result = df_detailed.loc[:, ['Season', 'DayNum', 'WTeamID', 'LTeamID']]
df_result['TeamID1'] = df_result.apply(lambda row: sorted([row['WTeamID'], row['LTeamID']])[0], axis=1)
df_result['TeamID2'] = df_result.apply(lambda row: sorted([row['WTeamID'], row['LTeamID']])[1], axis=1)
df_result['Pred'] = df_result['TeamID1'] == df_result['WTeamID']
df_result['Pred'] = df_result['Pred'].astype(float)
del df_result['WTeamID']
del df_result['LTeamID']
df_result.head()

### Coaches
We had access to coaching information which we thought would be useful as there are well known coaches who have been around for a long time and win a lot (which makes sense, they get paid to win). However after writing this section and adding the coaches to the model it actually caused the model to perform worse so it was taken out of the final model. It's been left in for historical purposes.

This is likely because the resulting feature set was so sparse from all of the one hot encoding that was performed.

In [ ]:
df_coaches = pd.read_csv(os.path.join(data_dir, 'Prelim2019_TeamCoaches.csv'))
df_coaches.head()

In [ ]:
# Which iteration the coach as been with the team
# Team A -> Team B -> Team A = 1 -> 2 -> 3

df_coaches = df_coaches.sort_values(by=['CoachName', 'Season'])
df_coaches['TeamNumber'] = df_coaches['TeamID'].ne(df_coaches['TeamID'].shift().bfill()).astype(int)
df_coaches['TeamNumber'] = df_coaches.groupby(['CoachName'])['TeamNumber'].cumsum()
df_coaches.head(30)

In [ ]:
# Handle special scenario where some coaches start with team number 0 instead of 1

team_number_0_coaches = df_coaches[df_coaches['TeamNumber']==0]['CoachName'].unique()
row_filter = df_coaches['CoachName'].isin(team_number_0_coaches)
df_coaches.loc[row_filter, 'TeamNumber'] = df_coaches.loc[row_filter, 'TeamNumber'] + 1

In [ ]:
df_coaches.head()

In [ ]:
# Calculate the game identifier
# This is required for the submission file
def calc_gameid(row):
    min_id = min(row['TeamID1'], row['TeamID2'])
    max_id = max(row['TeamID1'], row['TeamID2'])
    season = row['Season']
    return f'{int(season)}_{int(min_id)}_{int(max_id)}'

In [ ]:
df_c_pg = df_c_stats.loc[:, ['W', 'L', 'GameNum', 'DayNum', 'FG_PCT',
       'FG3_PCT', 'FT_PCT', 'Score_PG', 'OR_PG', 'DR_PG', 'Ast_PG', 'TO_PG',
       'Stl_PG', 'Blk_PG', 'PF_PG']]

mask = (df_c_pg['GameNum'] == 1)
df0_i = df_c_pg[mask].copy().index
df0 = pd.DataFrame(columns=df_c_pg.columns, index=df0_i)
df0 = df0.fillna(0)
df_c = df_c_pg.append(df0).reset_index()
df_c.head()

df_c_pg.head()

Next we need to shift certain data elements because currently it's tracking statistics with the current game included. When we predict a game in the future we won't know the stats for the game that hasn't been played yet. To remedy this we shift the data down to the next row so that the statistics stored are actually up to date for the start of the game rather than the end of the game.

In [ ]:
for shift_col in ['W', 'L', 'FG_PCT',
       'FG3_PCT', 'FT_PCT', 'Score_PG', 'OR_PG', 'DR_PG', 'Ast_PG', 'TO_PG',
       'Stl_PG', 'Blk_PG', 'PF_PG']:
    #df_c_pg[shift_col] = df_c_pg[shift_col].shift(1)
    df_c_pg[shift_col] = df_c_pg.groupby(level=[0,1])[shift_col].shift(1)

df_c_pg = df_c_pg.dropna()
df_c_pg.head()

In [ ]:
df_c_pg_coaches = pd.merge(
    left = df_c_pg.reset_index(),
    right = df_coaches,
    on = ['TeamID', 'Season']
)

# Filter out coaches based on the game day to handle coaches who are replaced mid-season
mask = (df_c_pg_coaches['FirstDayNum'] <= df_c_pg_coaches['DayNum'])
mask = mask & (df_c_pg_coaches['LastDayNum'] >= df_c_pg_coaches['DayNum'])
df_c_pg_coaches = df_c_pg_coaches[mask]
df_c_pg_coaches.head()

In [ ]:
# Calculate the number of seasons the coach has been coaching

df_c_pg_coaches['SeasonsCoaching'] = df_c_pg_coaches['Season'] - df_c_pg_coaches.groupby('CoachName')['Season'].transform('min')
del df_c_pg_coaches['FirstDayNum']
del df_c_pg_coaches['LastDayNum']
df_c_pg_coaches.head(30)

In [ ]:
# One hot encode the coach names
one_hot = pd.get_dummies(df_c_pg_coaches['CoachName'])
df_c_pg_coaches = df_c_pg_coaches.drop('CoachName', axis=1)
df_c_pg_coaches = df_c_pg_coaches.join(one_hot)
df_c_pg_coaches.head()

Merge the statistics with all of the games being played for both the winning and losing teams

In [ ]:
df_result_team1_details = pd.merge(
    left = df_c_pg, #df_c_pg_coaches,
    right = df_result,
    left_index = False,
    left_on = ['Season', 'DayNum', 'TeamID'],
    right_index = False,
    right_on = ['Season', 'DayNum', 'TeamID1']
)
df_result_team1_details.head()

df_result_team_details = pd.merge(
    left = df_c_pg, #df_c_pg_coaches,
    right = df_result_team1_details,
    left_index = False,
    left_on = ['Season', 'DayNum', 'TeamID'],
    right_index = False,
    right_on = ['Season', 'DayNum', 'TeamID2'],
    suffixes = ('_Team1', '_Team2')
)

df_result_team_details.head()

# Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
columns = [c for c in df_result_team_details if c not in ['TeamID1', 'TeamID2', 'DayNum']]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

For our model we only wanted to look at the 2019 data since college teams tend to have high turn over.

We're also scaling the data to make sure everything is on the same scale since the number of points per game will typically be much higher than the number of turnovers or assists per game.

In [ ]:
df_master = df_result_team_details.copy()
X = df_result_team_details[columns]
X = X[X['Season']==2019]
y = X.pop('Pred')
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8675309)

In [ ]:
lr = LogisticRegression(solver='lbfgs', max_iter=10000)

In [ ]:
lr.fit(X_train, y_train)
p = lr.predict_proba(X_test).clip(0.0000001, 0.99999999)
print('Log Loss:', log_loss(y_test, p))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
c = [x / 1000.0 for x in range(1, 1000, 1)]
# specify parameters and distributions to sample from
param_dist = param_grid = {'C': c, 'solver':['lbfgs'], 'max_iter':[100000]}

# run randomized search
clf = GridSearchCV(lr, param_grid, cv=10, n_jobs=-1)

In [ ]:
clf.fit(X, y)
clf.best_params_

In [ ]:
lr = LogisticRegression(**clf.best_params_)

In [ ]:
lr.fit(X_train, y_train)
p = lr.predict_proba(X_test).clip(0.0000001, 0.99999999)
print('Log Loss:', log_loss(y_test, p))

### Submission File

In [ ]:
df_result_team_details.head()

We don't want to utilize data from mid-season so here we're filtering to pull the last game from the season for each team.

In [ ]:
df_t = df_c_pg.reset_index()
idx = df_t.groupby(['TeamID', 'Season'])['GameNum'].idxmax()
df_t = df_t.iloc[idx]

In [ ]:
# Filter to only compare teams in the tournament this year
df_tourney_seeds = pd.read_csv(os.path.join(data_dir, 'NCAATourneySeeds.csv'))
df_tourney_seeds = df_tourney_seeds.loc[df_tourney_seeds['Season']==2019]
df_tourney_teams = df_tourney_seeds[['TeamID']]

df_t = pd.merge(
    left = df_t[df_t['Season']==2019],
    right = df_tourney_teams,

    on = ['TeamID'])

In [ ]:
# Join the DataFrame to itself to get all possible permutations
df_t_r = pd.merge(
    left = df_t, #df_c_pg_coaches,
    right = df_t,
    left_index = False,
    on = ['Season'],
    right_index = False,
    suffixes = ('_Team1', '_Team2')
)

In [ ]:
# Rename columns and move it to the end to match the same format
# that we used to train our model
df_t_r['TeamID1'] = df_t_r['TeamID_Team1']
df_t_r['TeamID2'] = df_t_r['TeamID_Team2']
del df_t_r['TeamID_Team1']
del df_t_r['TeamID_Team2']

# Since we can have 123 v 321 and 321 v 123 which would be
# the same just displayed differently and the competition
# wants just the lower team ID as team 1 let's de-dupe
df_t_r = df_t_r.loc[df_t_r['TeamID1'] < df_t_r['TeamID2'], :]

In [ ]:
df_t_r['GameID'] = df_t_r.apply(lambda row: calc_gameid(row), axis=1)

In [ ]:
gids = df_t_r.pop('GameID')


In [ ]:
# Hack
df_t_r['Pred'] = None
df_t_r = df_t_r[columns]
del df_t_r['Pred']

In [ ]:
X = df_t_r
X = scaler.fit_transform(X)
p = lr.predict_proba(X).clip(0.0000001, 0.99999999)
#print('Log Loss:', log_loss(y_test, p))

In [ ]:
pred = [prob[0] for prob in p]

In [ ]:
df_sub = pd.DataFrame({'ID':gids, 'Pred':pred})
df_sub.head()

In [ ]:
df_teams = pd.read_csv(os.path.join(data_dir, 'Teams.csv'))

In [ ]:
df_seeds = pd.read_csv(os.path.join(data_dir, 'NCAATourneySeeds.csv'))
df_slots = pd.read_csv(os.path.join(data_dir, 'NCAATourneySlots.csv'))

In [ ]:
df_seeds.head()

In [ ]:
df_bracket = pd.concat([df_sub, df_sub['ID'].str.extract(r'(?P<Season>\d+)_(?P<TeamID1>\d+)_(?P<TeamID2>\d+)', expand=True)], axis=1)
df_bracket.head()

In [ ]:
len(df_sub.index)

In [ ]:
df_bracket['TeamID1'] = df_bracket['TeamID1'].astype(int)
df_bracket['TeamID2'] = df_bracket['TeamID2'].astype(int)

In [ ]:
# Simple function to help fill out my bracket
def get_winner(team1, team2):
    print(team1, team2)
    if team1 > team2:
        t = team2
        team2 = team1
        team1 = t
    print(team1, team2)
    mask = df_bracket['TeamID1'] == team1
    mask = mask & (df_bracket['TeamID2'] == team2)
    pred = df_bracket.loc[mask, 'Pred'].values[0]
    if pred < 0.5:
        print(f'{team2} has a {(1-pred)*100}% chance of winning')
    else:
        print(f'{team1} has a {pred*100}% chance of winning')

In [ ]:
get_winner(1397, 1211)

In [ ]:
df_sub.to_csv('submission.csv', index=False)

# TODO:
* [X] calculate team wins and losses rolling for the season
  * Added Game 0 Values, just need to determine how to shift W/L columns down to show values coming into game
* [X] calculate rolling detailed stats FG %, FT %, 3PT %
  * [X] shift to be stats prior to game?
  * How handle first game of season as shift will make all stats NaN? Ignore first game of season.
* [X] DO NOT USE SCORE (We won't know it when predicting future games. We can use the season average score though)
* [X] Fine tune coach tenure (first season games are all currently 154 but it should be 0, 7, 14, 21 etc)
* [X] One hot encode coaches?
* [X] Shift per group not over all dataframe
* [X] Create Submission
* [ ] Clean up notebook for presentation
* * [X] Make note how adding coaches did worse because it was so sparse with OHE
* * [X] Explain Log-Loss function
* * * [X] Add Graph of Log-Loss with 50%/coin flip vertical line
* * [ ] Compare to picking the higher ranked teams
* * [ ] Compare to another bracket
* [X] Get Final 2018 dataset and rerun with 2019 data

# How'd we do?

# Ideas to improve the model next time:
* Rather than use the actual stats use a difference between the teams
* Utilize the play in tournament data
* Automatically generate bracket